In [ ]:
%matplotlib inline


# P300 Load and Visualize Data

This example demonstrates loading, organizing, and visualizing ERP response data from the visual P300 experiment. The experiment uses a visual oddball paradigm. Images of cats and dogs are shwn in a rapid serial visual presentation (RSVP) stream, with cats and dogs categorized respectively as 'targets' or 'non-targets', according to which has high or low probability of occurring, respectively. 

The data used is the first subject and first session of the one of the eeg-notebooks P300 example datasets, recorded using the InteraXon MUSE EEG headset (2016 model). This session consists of six two-minute blocks of continuous recording.  

We first use the `fetch_datasets` to obtain a list of filenames. If these files are not already present 
in the specified data directory, they will be quickly downloaded from the cloud. 

After loading the data, we place it in an MNE `Epochs` object, and obtain the trial-averaged response. 

The final figure plotted at the end shows the P300 response ERP waveform. 


## Setup



In [13]:
# Some standard pythonic imports
import os
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

from typing import Union

import requests, zipfile, gdown

# MNE functions
from mne import Epochs,find_events

# EEG-Notebooks functions
# from eegnb.analysis.utils import load_data,plot_conditions
# from eegnb.datasets import fetch_dataset

ModuleNotFoundError: No module named 'eegnb'

# Load eegnb: `fetch_dataset` and `load_data`

In [14]:
# eegnb example data sites. do not select these when zipping recordings
eegnb_sites = ['eegnb_examples', 'grifflab_dev', 'jadinlab_home']

def fetch_dataset(
    data_dir=None,
    experiment=None,
    site="eegnb_examples",
    device="muse2016",
    subjects="all",
    sessions="all",
    download_method="gdown",
):
    """
    Return a long-form filenames list and a table saying what
    subject and session, and run each entry corresponds to
    Usage:
            data_dir = '/my_folder'
            experiment = 'visual-N170'
            subjects = [1]
            sessions = 'all'
            visn170_fnames = fetch_dataset(data_dir=data_dir, subjects='all', experiment='visual-N170',
            site='eegnb_examples')
            visnP300_fnames = fetch_dataset(data_dir=data_dir, subjects=[1], experiment='visual-P300',
            site='eegnb_examples')
    """
    # List of experiments available
    experiments_list = [
        "rest",
        "auditory-P300",
        "auditory-SSAEP",
        "visual-cueing",
        "visual-gonogo",
        "visual-leftright",
        "visual-N170",
        "visual-P300",
        "visual-spatialfreq",
        "visual-SSVEP",
    ]

    # List gdrive extensions for various experiments
    gdrive_locs = {
        "visual-SSVEP": "1zj9Wx-YEMJo7GugUUu7Sshcybfsr-Fze",
        "visual-spatialfreq": "1ggBt7CNvMgddxji-FvxcZoP-IF-PmESX",
        "visual-P300": "1OLcj-zSjqdNrsBSUAsGBXOwWDnGWTVFC",
        "visual-N170": "1oStfxzEqf36R5d-2Auyw4DLnPj9E_FAH",
        "visual-leftright": "1f8A4Vbz0xjfgGIYFldMZ7ZL02x7T0jSt",
        "visual-nogono": "1C8WKg9TXyp8A3QJ6T8zbGnk6jFcMutad",
        "visual-cueing": "1ABOVJ9S0BeJOsqdGFnexaTFZ-ZcsIXfQ",
        "auditory-SSAEP": "1fd0OAyNGWWOHD8e1FnEOLeQMeEoxqEpO",
        "auditory-P300": "1OEtrRfMOkzDssGv-2Lj56FsArmPnQ2vD",
    }

    # If no non-default top-level data path specified, use default
    if data_dir == None:
        data_dir = DATA_DIR

    # check parameter entries
    if experiment not in experiments_list:
        raise ValueError("experiment not in database")

    # check if data has been previously downloaded
    download_it = False
    exp_dir = os.path.join(data_dir, experiment, site, device)
    if not os.path.isdir(exp_dir):
        download_it = True

    if download_it:
        # check if data directory exits. If not, create it
        if os.path.exists(data_dir) is not True:
            os.makedirs(data_dir)

        destination = os.path.join(data_dir, "downloaded_data.zip")

        if download_method == "gdown":

            URL = "https://drive.google.com/uc?id=" + gdrive_locs[experiment]
            gdown.download(URL, destination, quiet=False)

        elif download_method == "requests":

            URL = "https://docs.google.com/uc?export=download"

            session = requests.Session()
            response = session.get(
                URL, params={"id": gdrive_locs[experiment]}, stream=True
            )

            # get the confirmation token to download large files
            token = None
            for key, value in response.cookies.items():
                if key.startswith("download_warning"):
                    token = value

            if token:
                params = {"id": id, "confirm": token}
                response = session.get(URL, params=params, stream=True)

            # save content to the zip-file
            CHUNK_SIZE = 32768
            with open(destination, "wb") as f:
                for chunk in response.iter_content(CHUNK_SIZE):
                    if chunk:
                        f.write(chunk)

        # unzip the file
        with zipfile.ZipFile(destination, "r") as zip_ref:
            zip_ref.extractall(data_dir)

        # remove the compressed zip archive
        os.remove(destination)

    if subjects == "all":
        subjects = ["*"]
    if sessions == "all":
        sessions = ["*"]

    # If 'all' subjects and 'all sesssions:
    if (subjects[0] == "*") and (sessions[0] == "*"):
        pth = os.path.join(
            exp_dir, f"subject{subjects[0]}", f"session{sessions[0]}", "*.csv"
        )
        fnames = glob.glob(pth)
    # Else, if specific subjects and sessions
    else:
        fnames = []
        for subject_nb in subjects:
            if subject_nb != "*":
                # Format to get 4 digit number, e.g. 0004
                subject_nb = float(subject_nb)
                subject_nb = "%03.f" % subject_nb
                for session_nb in sessions:
                    # Formt to get 3 digit number, e.g. 003
                    if session_nb != "*":
                        session_nb = float(session_nb)
                        session_nb = "%02.f" % session_nb

                        pth = os.path.join(
                            exp_dir,
                            f"subject{subject_nb}",
                            f"session{session_nb}",
                            "*.csv",
                        )
                        # pth = '{}/subject{}/session{}/*.csv'.format(exp_dir,subject_nb, session_nb)
                        fpaths = glob.glob(pth)
                        fnames += fpaths

    return fnames

def load_data(
    subject_id: Union[str, int],
    session_nb: Union[str, int],
    device_name: str,
    experiment: str,
    replace_ch_names=None,
    verbose=1,
    site="local",
    data_dir=None,
    inc_chans=None,
) -> RawArray:
    """Load CSV files from the /data directory into a Raw object.
    This is a utility function that simplifies access to eeg-notebooks
    recordings by wrapping `load_csv_as_raw()`.
    The provided information is used to recover an eeg-notebooks recording file
    path with the following structure:
    data_dir/experiment/site/device_name/subject_str/session_str/<recording_date_time>.csv'
    where <recording_date_time> is the automatically generated file name(s)
    given at the time of recording.
    Args:
        subject_id (int or str): subject number. If 'all', load all
            subjects.
        session_nb (int or str): session number. If 'all', load all
            sessions.
        device_name (str): name of device. For a list of supported devices, see
            eegnb.analysis.utils.SAMPLE_FREQS.
        experiment (int or str): experiment name or number.
        inc_chans (array_like): (Optional) Selective list of the number of the
            channels to be imported
    Keyword Args:
        replace_ch_names (dict or None): dictionary containing a mapping to
            rename channels. Useful when e.g., an external electrode was used.
        verbose (int): verbose level.
        site (str): site of recording. If 'all', data from all sites will be
            used.
        data_dir (str or None): directory inside /data that contains the
            CSV files to load, e.g., 'auditory/'.
    Returns:
        (mne.io.RawArray): loaded EEG
    """

    subject_str = "*" if subject_id == "all" else f"subject{subject_id:04}"
    session_str = "*" if session_nb == "all" else f"session{session_nb:03}"
    if site == "all":
        site = "*"

    data_path = (
        _get_recording_dir(device_name, experiment, subject_str, session_str, site, data_dir)
        / "*.csv"
    )
    fnames = glob(str(data_path))

    sfreq = SAMPLE_FREQS[device_name]

    ch_ind = EEG_INDICES[device_name]

    if inc_chans is not None:
        ch_ind = inc_chans

    if device_name in ["muse2016", "muse2", "museS"]:
        return load_csv_as_raw(
            fnames,
            sfreq=sfreq,
            ch_ind=ch_ind,
            aux_ind=[5],
            replace_ch_names=replace_ch_names,
            verbose=verbose,
        )
    else:
        return load_csv_as_raw(
            fnames,
            sfreq=sfreq,
            ch_ind=ch_ind,
            replace_ch_names=replace_ch_names,
            verbose=verbose,
        )

NameError: name 'RawArray' is not defined

Load Data
 ---------------------

 We will use the eeg-notebooks N170 example dataset

 Note that if you are running this locally, the following cell will download
 the example dataset, if you do not already have it.

##################################################################################################



In [12]:
eegnb_data_path = os.path.join(os.path.expanduser('~/'),'eegnb', 'data')    
p300_data_path = os.path.join(eegnb_data_path, 'visual-P300', 'eegnb_examples')

# If dataset hasn't been downloaded yet, download it 
if not os.path.isdir(p300_data_path):
    fetch_dataset(data_dir=eegnb_data_path, experiment='visual-P300', site='eegnb_examples');        


subject = 1
session = 1
raw = load_data(subject,session,
                experiment='visual-P300', site='eegnb_examples', device_name='muse2016',
                data_dir = eegnb_data_path)

Downloading...
From: https://drive.google.com/uc?id=1OLcj-zSjqdNrsBSUAsGBXOwWDnGWTVFC
To: /Users/phily/eegnb/data/downloaded_data.zip
100%|██████████| 18.4M/18.4M [00:01<00:00, 16.0MB/s]


NameError: name 'load_data' is not defined

## Visualize the power spectrum



In [ ]:
raw.plot_psd()

## Filteriing



In [ ]:
raw.filter(1,30, method='iir')
raw.plot_psd(fmin=1, fmax=30);

## Epoching



In [ ]:
# Create an array containing the timestamps and type of each stimulus (i.e. face or house)
events = find_events(raw)
event_id = {'Non-Target': 1, 'Target': 2}
epochs = Epochs(raw, events=events, event_id=event_id,
                tmin=-0.1, tmax=0.8, baseline=None,                                                                                  
                reject={'eeg': 100e-6}, preload=True,                                                                                  
                verbose=False, picks=[0,1,2,3])

print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)

epochs

## Epoch average



In [ ]:
conditions = OrderedDict()
conditions['Non-target'] = [1]
conditions['Target'] = [2]

fig, ax = plot_conditions(epochs, conditions=conditions, 
                          ci=97.5, n_boot=1000, title='',
                          diff_waveform=(1, 2))